# Co je to Nanopore sekvenování?

## Jak to funguje?

- Nanopór je maličký otvor v tenké membráně.
- Skrz nanopór protéká elektrolyt a tím i slabý elektrický proud.
- Když DNA prochází skrz pór, **každý DNA nukleotid (A, T, C, G)** trochu změní proud. Každý jiným způsobem.
- Tato změna se zaznamenává jako **průběh signálu**.
- Počítač pak z tohoto signálu pozná, jaká písmena DNA šla dovnitř.

Podívej se na krátké video (2 minuty):  
🔗 [Jak funguje nanopórové sekvenování – od Oxford Nanopore](https://www.youtube.com/watch?v=2C9gRz8OTR8)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("RcP85JHLmnI", width=800, height=450)

## Podíváme se na skutečný signál

V následujícím grafu uvidíme, jak vypadá **surový signál** ze sekvenátoru.

In [ ]:
import pod5
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode()

# načteme ready z .pod5 souboru
file_path = "temp.pod5"
with pod5.Reader(file_path) as reader:
    read = next(reader.reads())
    signal = read.signal

# vykreslíme surový signál
plt.figure(figsize=(14, 4))
plt.plot(signal[:300])  # Plot first 300 signal points
plt.title("Hrubý signál jednoho čtení z Nanopore")
plt.xlabel("Čas")
plt.ylabel("Proud (pA)")
plt.grid(True)
plt.show()

### Otázky na zamyšlení:

**Proč se mění signál, když DNA prochází nanopórem?**

<details>
<summary>Odpověď</summary>

Každé písmeno DNA má jiný tvar a jinou velikost, takže ovlivňuje proud jinak. Když DNA prochází nanopórem, ovlivňuje proud iontů (např. Na⁺, Cl⁻), který jím prochází. Různé kombinace písmen DNA (tzv. k-mery) mají různý tvar a elektrické vlastnosti, takže každý z nich mění proud jiným způsobem. Počítač pak z těchto změn pozná, která písmena tam byla.

![DNA nukleotidy](nucleotide-bases-from-structure-to-modifications-1.jpg)
</details>

**Když dostaneme tento signál, jak bys z něj poznal, jaká písmena jsou v DNA?**

<details>
<summary>Odpověď</summary>
Počítač se to učí z tisíců příkladů pomocí umělé inteligence.

**Zkus si představit:**
Kdybys měl z poslechu hluku určit, co jede po silnici (auto, kolo, autobus), dokážeš to? Nanopór to dělá podobně – rozpoznává, co právě prošlo.
</details>

## Převod signálu na DNA sekvenci – Basecalling

Když molekula DNA prochází nanopórem, zaznamenáváme změny elektrického proudu. Tyto změny tvoří tzv. **surový signál** (anglicky *raw signal*), který je uložen v souborech typu `.pod5`.

Ale abychom zjistili, jaká písmena DNA (A, T, C, G) odpovídají tomuto signálu, potřebujeme **převodník** – ten se nazývá **basecaller**.

## Co dělá basecaller?

- Analyzuje průběh elektrického signálu
- Pomocí umělé inteligence a trénovaných modelů odhadne, jaké nukleotidy procházely nanopórem
- Výsledek je sekvence DNA ve formátu **FASTQ** – tedy:
  - Sekvence (např. `AGGCTTAC...`)
  - Kvalita každého písmena (číslem vyjádřená jistota)

Pro převod použijeme nástroj **Dorado** od Oxford Nanopore Technologies.

V další části nejprve ukážeme basecalling jednotlivých čtení v jednom .pod5 souboru, a poté zpracujeme všechna najednou.

In [ ]:
# (beží asi 15 minut)
# !python3 print_basecalled.py /home/lab33/researchers_night/dna_r10.4.1_e8.2_400bps_sup@v5.0.0/ temp.pod5 # TODO upravit cestu

### Co je ve FASTQ souboru?

Soubor `.fastq` obsahuje výsledky sekvenování – tedy čtení DNA. Každé čtení má 4 řádky:

1. `@` a název čtení
2. Sekvence DNA (např. `ACGTTG...`)
3. `+` jako oddělovač
4. Kvalita jednotlivých bází (čím vyšší ASCII znak, tím vyšší jistota)

In [ ]:
!zcat /var/lib/minknow/data/2025_07_15_NUVR_RB0/no_sample_id/20250715_1449_P2S-02648-B_PBC42512_57bb6767/fastq_pass/barcode06/PBC42512_pass_barcode06_57bb6767_39d08f64_0.fastq.gz | head -n 24 # TODO upravit cestu

### Zkus si: K jakému organismu patří náš vzorek?

1. Vyber jednu ze sekvencí výše
2. Zkopíruj ji a vlož do nástroje [NCBI BLAST (nucleotide)](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROGRAM=blastn)
3. Stiskni "BLAST"
4. Sleduj, jak databáze najde podobné sekvence a řekne ti, odkud pochází

### Zamysli se

- Jsou některé čtení delší než jiné?
- Všimni si, že některá čtení mají horší kvalitu (nižší znaky ve 4. řádku), použij [tabulku vysvětlující  Phred skóre](https://en.wikipedia.org/wiki/Phred_quality_score)
- Co se asi stane, když pošleme do BLASTu krátké vs. dlouhé čtení?

<details>
<summary>Odpověď</summary>

- Ano, délka čtení se může lišit. Nanopore umožňuje číst velmi dlouhé úseky DNA, ale některá čtení mohou být přerušena dříve.
- Znaky ve 4. řádku ukazují kvalitu každé báze. Nižší znaky znamenají nižší jistotu – mohou být způsobeny šumem v signálu.
- Krátká čtení mají menší šanci najít jednoznačný výsledek v BLASTu – mohou se hodit k více oblastem v různých genomech. Delší čtení jsou často specifičtější a dají lepší nápovědu o původu vzorku.

</details>

## Jak se liší genetická informace zdravého člověka a onkopacienta?

### Co je to karyotyp a proč ho zkoumáme?

Každá lidská buňka (kromě pohlavních buněk) obsahuje 46 chromozomů – to je náš karyotyp. Chromozomy jsou "balíčky" DNA, které nesou naši genetickou informaci. Zdravé buňky mají kompletní a správně uspořádaný karyotyp – 23 párů chromozomů, žádné navíc, žádné chybějící části.

U nádorových buněk ale často dochází ke změnám – mohou mít některé chromozomy zdvojené, ztracené, přeskupené nebo přerušené. Takové změny označujeme jako chromozomální aberace, a jsou jedním z typických znaků rakovinných buněk. Právě proto se karyotyp nádorových buněk zkoumá – může nám napovědět, co je v buňce špatně a jak rakovina vznikla.

![Lidský karyotyp](Figure_13_03_01.jpg)

### Jak to zjistíme pomocí sekvenování

Pomocí nanopórového sekvenování DNA získáme dlouhé náhodně vybrané úseky genetické informace z jednotlivých chromozomů. Výsledkem je obrovské množství "čtení" – krátkých úseků DNA, které se zpětně zarovnají na jednotlivé chromozomy lidského genomu.

Při analýze sledujeme tzv. pokrytí (coverage) – tedy kolik čtení se mapuje na jednotlivé části každého chromozomu. Pokud má nějaký chromozom vyšší pokrytí, může to znamenat, že je v buňce zdvojený. Naopak nižší pokrytí může naznačovat ztrátu části nebo celého chromozomu.

### Proč porovnáváme zdravé a nádorové buňky?

Porovnáním karyotypu zdravých pacientů a onkopacientů můžeme zjistit, které chromozomy nebo jejich části jsou v nádorových buňkách změněné. To nám pomáhá pochopit, co se v buňkách děje při vzniku rakoviny, a může to být i důležitý krok ke zlepšení diagnostiky nebo léčby.

### Jak získáme pokrytí jednotlivých chromozomů?

Abychom mohli porovnat karyotyp zdravého a nemocného člověka, potřebujeme nejprve jejich sekvenovanou DNA zarovnat na referenční lidský genom – tedy na "mapu" správně uspořádané lidské DNA, kterou vědci sestavili.

Zarovnání na referenci: Pomocí bioinformatických nástrojů (např. minimap2) zarovnáme všechna přečtení DNA (tzv. "reads") ke správnému místu v lidském genomu.


In [ ]:
# nastavíme cesty k souborům
# TODO upravit cesty k souborům
test1 = "/var/lib/minknow/data/2025_07_15_NUVR_RB0/no_sample_id/20250715_1449_P2S-02648-B_PBC42512_57bb6767/fastq_pass/barcode06"
test2 = "/var/lib/minknow/data/2025_07_15_NUVR_RB0/no_sample_id/20250715_1449_P2S-02648-B_PBC42512_57bb6767/fastq_pass/barcode08"

healthy_control_rep1_fastq = test1
cancer_sample1_rep1_fastq = test2
cancer_sample2_rep1_fastq = test2

healthy_control_rep2_fastq = test1
cancer_sample1_rep2_fastq = test1
cancer_sample2_rep2_fastq = test1

healthy_control_rep3_fastq = test2
cancer_sample1_rep3_fastq = test2
cancer_sample2_rep3_fastq = test2

reference_genome = "Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

In [ ]:
# spojíme všechny fastq soubory stejného vzorku do jednoho souboru
# TODO upravit cesty k souborum
!cat "$healthy_control_rep1_fastq"/PBC42512_pass_barcode06_57bb6767_39d08f64_*.fastq.gz > healthy_control_rep1.fastq.gz
!cat "$cancer_sample1_rep1_fastq"/PBC42512_pass_barcode08_57bb6767_39d08f64_*.fastq.gz > cancer_sample1_rep1.fastq.gz
!cat "$cancer_sample2_rep1_fastq"/PBC42512_pass_barcode08_57bb6767_39d08f64_*.fastq.gz > cancer_sample2_rep1.fastq.gz

!cat "$healthy_control_rep2_fastq"/PBC42512_pass_barcode06_57bb6767_39d08f64_*.fastq.gz > healthy_control_rep2.fastq.gz
!cat "$cancer_sample1_rep2_fastq"/PBC42512_pass_barcode06_57bb6767_39d08f64_*.fastq.gz > cancer_sample1_rep2.fastq.gz
!cat "$cancer_sample2_rep2_fastq"/PBC42512_pass_barcode06_57bb6767_39d08f64_*.fastq.gz > cancer_sample2_rep2.fastq.gz

!cat "$healthy_control_rep3_fastq"/PBC42512_pass_barcode08_57bb6767_39d08f64_*.fastq.gz > healthy_control_rep3.fastq.gz
!cat "$cancer_sample1_rep3_fastq"/PBC42512_pass_barcode08_57bb6767_39d08f64_*.fastq.gz > cancer_sample1_rep3.fastq.gz
!cat "$cancer_sample2_rep3_fastq"/PBC42512_pass_barcode08_57bb6767_39d08f64_*.fastq.gz > cancer_sample2_rep3.fastq.gz

In [ ]:
# namapujeme čtení na referenční lidský genom
!minimap2 -ax map-ont -t 12 "$reference_genome" healthy_control_rep1.fastq.gz | samtools sort -o healthy_control_rep1.sorted.bam && samtools index healthy_control_rep1.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample1_rep1.fastq.gz | samtools sort -o cancer_sample1_rep1.sorted.bam && samtools index cancer_sample1_rep1.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample2_rep1.fastq.gz | samtools sort -o cancer_sample2_rep1.sorted.bam && samtools index cancer_sample2_rep1.sorted.bam

!minimap2 -ax map-ont -t 12 "$reference_genome" healthy_control_rep2.fastq.gz | samtools sort -o healthy_control_rep2.sorted.bam && samtools index healthy_control_rep2.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample1_rep2.fastq.gz | samtools sort -o cancer_sample1_rep2.sorted.bam && samtools index cancer_sample1_rep2.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample2_rep2.fastq.gz | samtools sort -o cancer_sample2_rep2.sorted.bam && samtools index cancer_sample2_rep2.sorted.bam

!minimap2 -ax map-ont -t 12 "$reference_genome" healthy_control_rep3.fastq.gz | samtools sort -o healthy_control_rep3.sorted.bam && samtools index healthy_control_rep3.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample1_rep3.fastq.gz | samtools sort -o cancer_sample1_rep3.sorted.bam && samtools index cancer_sample1_rep3.sorted.bam
!minimap2 -ax map-ont -t 12 "$reference_genome" cancer_sample2_rep3.fastq.gz | samtools sort -o cancer_sample2_rep3.sorted.bam && samtools index cancer_sample2_rep3.sorted.bam

In [ ]:
# vyfiltrujeme pouze kvalitně namapovaná čtení
!samtools view -F 2308 healthy_control_rep1.sorted.bam -o healthy_control_rep1.primary_mapped.sorted.bam && samtools index healthy_control_rep1.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample1_rep1.sorted.bam -o cancer_sample1_rep1.primary_mapped.sorted.bam && samtools index cancer_sample1_rep1.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample2_rep1.sorted.bam -o cancer_sample2_rep1.primary_mapped.sorted.bam && samtools index cancer_sample2_rep1.primary_mapped.sorted.bam

!samtools view -F 2308 healthy_control_rep2.sorted.bam -o healthy_control_rep2.primary_mapped.sorted.bam && samtools index healthy_control_rep2.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample1_rep2.sorted.bam -o cancer_sample1_rep2.primary_mapped.sorted.bam && samtools index cancer_sample1_rep2.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample2_rep2.sorted.bam -o cancer_sample2_rep2.primary_mapped.sorted.bam && samtools index cancer_sample2_rep2.primary_mapped.sorted.bam

!samtools view -F 2308 healthy_control_rep3.sorted.bam -o healthy_control_rep3.primary_mapped.sorted.bam && samtools index healthy_control_rep3.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample1_rep3.sorted.bam -o cancer_sample1_rep3.primary_mapped.sorted.bam && samtools index cancer_sample1_rep3.primary_mapped.sorted.bam
!samtools view -F 2308 cancer_sample2_rep3.sorted.bam -o cancer_sample2_rep3.primary_mapped.sorted.bam && samtools index cancer_sample2_rep3.primary_mapped.sorted.bam

#### Jak vypadají namapovaná čtení?

In [ ]:
# SPUSTIT IGV DESKTOP

Výpočet pokrytí (coverage): Poté spočítáme, kolik čtení připadá na každý chromozom nebo jeho část. Čím více přečtení na určitém místě, tím vyšší pokrytí.

Vykreslení grafu: Pokrytí zobrazíme v grafu – každý chromozom bude mít svůj sloupec nebo bod. Zdravý vzorek by měl mít zhruba rovnoměrné pokrytí všech chromozomů. Nádorový vzorek ale může mít výrazné výkyvy – některé chromozomy s vyšším nebo nižším pokrytím, což ukazuje na změny v počtu kopií chromozomů (copy number variations).

## Vykreslíme a porovnáme pokrytí jednotlivých chromozomů

Porovnáme pokrytí celých chromozomů.

In [ ]:
%%bash
samtools coverage healthy_control_rep1.primary_mapped.sorted.bam | head -n 25 > healthy_control_rep1.coverage.tsv
samtools coverage cancer_sample1_rep1.primary_mapped.sorted.bam | head -n 25 > cancer_sample1_rep1.coverage.tsv
samtools coverage cancer_sample2_rep1.primary_mapped.sorted.bam | head -n 25 > cancer_sample2_rep1.coverage.tsv
samtools coverage healthy_control_rep2.primary_mapped.sorted.bam | head -n 25 > healthy_control_rep2.coverage.tsv
samtools coverage cancer_sample1_rep2.primary_mapped.sorted.bam | head -n 25 > cancer_sample1_rep2.coverage.tsv
samtools coverage cancer_sample2_rep2.primary_mapped.sorted.bam | head -n 25 > cancer_sample2_rep2.coverage.tsv
samtools coverage healthy_control_rep3.primary_mapped.sorted.bam | head -n 25 > healthy_control_rep3.coverage.tsv
samtools coverage cancer_sample1_rep3.primary_mapped.sorted.bam | head -n 25 > cancer_sample1_rep3.coverage.tsv
samtools coverage cancer_sample2_rep3.primary_mapped.sorted.bam | head -n 25 > cancer_sample2_rep3.coverage.tsv

In [ ]:
healthy_control_rep1_coverage_df = pd.read_csv("healthy_control_rep1.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
healthy_control_rep1_coverage_df.rename(columns={"meandepth": "healthy_control_rep1_coverage"}, inplace=True)
cancer_sample1_rep1_coverage_df = pd.read_csv("cancer_sample1_rep1.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample1_rep1_coverage_df.rename(columns={"meandepth": "cancer_sample1_rep1_coverage"}, inplace=True)
cancer_sample2_rep1_coverage_df = pd.read_csv("cancer_sample2_rep1.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample2_rep1_coverage_df.rename(columns={"meandepth": "cancer_sample2_rep1_coverage"}, inplace=True)

healthy_control_rep2_coverage_df = pd.read_csv("healthy_control_rep2.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
healthy_control_rep2_coverage_df.rename(columns={"meandepth": "healthy_control_rep2_coverage"}, inplace=True)
cancer_sample1_rep2_coverage_df = pd.read_csv("cancer_sample1_rep2.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample1_rep2_coverage_df.rename(columns={"meandepth": "cancer_sample1_rep2_coverage"}, inplace=True)
cancer_sample2_rep2_coverage_df = pd.read_csv("cancer_sample2_rep2.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample2_rep2_coverage_df.rename(columns={"meandepth": "cancer_sample2_rep2_coverage"}, inplace=True)

healthy_control_rep3_coverage_df = pd.read_csv("healthy_control_rep3.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
healthy_control_rep3_coverage_df.rename(columns={"meandepth": "healthy_control_rep3_coverage"}, inplace=True)
cancer_sample1_rep3_coverage_df = pd.read_csv("cancer_sample1_rep3.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample1_rep3_coverage_df.rename(columns={"meandepth": "cancer_sample1_rep3_coverage"}, inplace=True)
cancer_sample2_rep3_coverage_df = pd.read_csv("cancer_sample2_rep3.coverage.tsv", sep="\t", usecols = ["#rname", "startpos", "endpos", "meandepth"])
cancer_sample2_rep3_coverage_df.rename(columns={"meandepth": "cancer_sample2_rep3_coverage"}, inplace=True)

In [ ]:
coverages_df = healthy_control_rep1_coverage_df.merge(cancer_sample1_rep1_coverage_df).merge(cancer_sample2_rep1_coverage_df).merge(healthy_control_rep2_coverage_df).merge(cancer_sample1_rep2_coverage_df).merge(cancer_sample2_rep2_coverage_df).merge(healthy_control_rep3_coverage_df).merge(cancer_sample1_rep3_coverage_df).merge(cancer_sample2_rep3_coverage_df)
coverages_df

#### Normalizace hloubky sekvenování

Po namapování čtení na referenční genom můžeme spočítat, kolikrát byla každá část DNA přečtena — tomu říkáme **pokrytí** (*coverage*). Ale...

##### Proč musíme pokrytí normalizovat?

Každý vzorek (nebo čárový kód) může mít **různý počet čtení**. Například:

- Vzorek A: 10 000 čtení
- Vzorek B: 100 000 čtení

Pokud bychom porovnávali jejich pokrytí přímo, zdálo by se, že vzorek B má "více DNA" — ale to je jen proto, že jsme ho více sekvenovali.

Abychom mohli porovnávat vzorky **spravedlivě**, musíme jejich pokrytí **přepočítat na jedno čtení**.

##### Jak to uděláme?

📌 Pro každý vzorek:
1. Spočítáme **pokrytí** (např. kolikrát byla přečtena každá část chromozomu).
2. Spočítáme, kolik bylo celkem čtení zarovnaných na celý genom.
3. Vydělíme pokrytí počtem čtení → tím dostaneme **normalizované pokrytí**.

Díky tomu můžeme porovnat, jestli má některý chromozom (nebo část genomu) vyšší nebo nižší pokrytí **nezávisle na tom, kolik čtení bylo celkem**.

To je užitečné například pro hledání **chromozomových aberací** — když je nějaký chromozom přítomen vícekrát nebo chybí.

In [ ]:
%%bash
# převedeme bam na fastq abychom mohli spočítat počet čtení na vzorek
samtools fastq healthy_control_rep1.primary_mapped.sorted.bam > healthy_control_rep1.primary_mapped.fastq
samtools fastq cancer_sample1_rep1.primary_mapped.sorted.bam > cancer_sample1_rep1.primary_mapped.fastq
samtools fastq cancer_sample2_rep1.primary_mapped.sorted.bam > cancer_sample2_rep1.primary_mapped.fastq

samtools fastq healthy_control_rep2.primary_mapped.sorted.bam > healthy_control_rep2.primary_mapped.fastq
samtools fastq cancer_sample1_rep2.primary_mapped.sorted.bam > cancer_sample1_rep2.primary_mapped.fastq
samtools fastq cancer_sample2_rep2.primary_mapped.sorted.bam > cancer_sample2_rep2.primary_mapped.fastq

samtools fastq healthy_control_rep3.primary_mapped.sorted.bam > healthy_control_rep3.primary_mapped.fastq
samtools fastq cancer_sample1_rep3.primary_mapped.sorted.bam > cancer_sample1_rep3.primary_mapped.fastq
samtools fastq cancer_sample2_rep3.primary_mapped.sorted.bam > cancer_sample2_rep3.primary_mapped.fastq

In [ ]:
from Bio import SeqIO

def get_read_count(fastq_gz_file):
    fastq_path = fastq_gz_file
    read_count = 0

    for _ in SeqIO.parse(fastq_gz_file, "fastq"):
        read_count += 1
    
    return read_count

In [ ]:
# spočítáme počet čtení na vzorek
healthy_control_rep1_num_reads = get_read_count("healthy_control_rep1.primary_mapped.fastq")
cancer_sample1_rep1_num_reads = get_read_count("cancer_sample1_rep1.primary_mapped.fastq")
cancer_sample2_rep1_num_reads = get_read_count("cancer_sample2_rep1.primary_mapped.fastq")

healthy_control_rep2_num_reads = get_read_count("healthy_control_rep2.primary_mapped.fastq")
cancer_sample1_rep2_num_reads = get_read_count("cancer_sample1_rep2.primary_mapped.fastq")
cancer_sample2_rep2_num_reads = get_read_count("cancer_sample2_rep2.primary_mapped.fastq")

healthy_control_rep3_num_reads = get_read_count("healthy_control_rep3.primary_mapped.fastq")
cancer_sample1_rep3_num_reads = get_read_count("cancer_sample1_rep3.primary_mapped.fastq")
cancer_sample2_rep3_num_reads = get_read_count("cancer_sample2_rep3.primary_mapped.fastq")

In [ ]:
# vydělíme tabulku pokrytí počtem čtení
multiply_by = 1_000_000
coverages_df["healthy_control_rep1_coverage"] = (coverages_df["healthy_control_rep1_coverage"] / healthy_control_rep1_num_reads) * multiply_by
coverages_df["cancer_sample1_rep1_coverage"] = (coverages_df["cancer_sample1_rep1_coverage"] / cancer_sample1_rep1_num_reads) * multiply_by
coverages_df["cancer_sample2_rep1_coverage"] = (coverages_df["cancer_sample2_rep1_coverage"] / cancer_sample2_rep1_num_reads) * multiply_by

coverages_df["healthy_control_rep2_coverage"] = (coverages_df["healthy_control_rep2_coverage"] / healthy_control_rep2_num_reads) * multiply_by
coverages_df["cancer_sample1_rep2_coverage"] = (coverages_df["cancer_sample1_rep2_coverage"] / cancer_sample1_rep2_num_reads) * multiply_by
coverages_df["cancer_sample2_rep2_coverage"] = (coverages_df["cancer_sample2_rep2_coverage"] / cancer_sample2_rep2_num_reads) * multiply_by

coverages_df["healthy_control_rep3_coverage"] = (coverages_df["healthy_control_rep3_coverage"] / healthy_control_rep3_num_reads) * multiply_by
coverages_df["cancer_sample1_rep3_coverage"] = (coverages_df["cancer_sample1_rep3_coverage"] / cancer_sample1_rep3_num_reads) * multiply_by
coverages_df["cancer_sample2_rep3_coverage"] = (coverages_df["cancer_sample2_rep3_coverage"] / cancer_sample2_rep3_num_reads) * multiply_by

In [ ]:
coverages_df

In [ ]:
fig = px.scatter(coverages_df, x="#rname", y=["healthy_control_rep1_coverage", "cancer_sample1_rep1_coverage", "cancer_sample2_rep1_coverage", "healthy_control_rep2_coverage", "cancer_sample1_rep2_coverage", "cancer_sample2_rep2_coverage", "healthy_control_rep3_coverage", "cancer_sample1_rep3_coverage", "cancer_sample2_rep3_coverage"], title="Pokrytí chromozomů")
fig.update_layout(xaxis_title="Chromozom #", yaxis_title="Pokrytí")
fig.update_traces(marker_size=10)
fig.show()

In [ ]:
coverages_df["healthy_controls_mean"] = coverages_df[["healthy_control_rep1_coverage", "healthy_control_rep2_coverage", "healthy_control_rep3_coverage"]].mean(axis=1) 
coverages_df["cancer_samples1_mean"] = coverages_df[["cancer_sample1_rep1_coverage", "cancer_sample1_rep2_coverage", "cancer_sample1_rep3_coverage"]].mean(axis=1)
coverages_df["cancer_samples2_mean"] = coverages_df[["cancer_sample2_rep1_coverage", "cancer_sample2_rep2_coverage", "cancer_sample2_rep3_coverage"]].mean(axis=1)

In [ ]:
fig = px.scatter(coverages_df, x="#rname", y=["healthy_controls_mean", "cancer_samples1_mean", "cancer_samples2_mean"], title="Pokrytí chromozomů zprůměrované přes replikáty")
fig.update_traces(marker_size=10)
fig.update_layout(xaxis_title="Chromozom #", yaxis_title="Pokrytí")
fig.show()

A nyní se podíváme na pokrytí zblízka.

In [ ]:
# spočítáme pokrytí podél chromozomů
# !bedtools genomecov -ibam healthy_control_rep1.primary_mapped.sorted.bam -bga > healthy_control_rep1.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample1_rep1.primary_mapped.sorted.bam -bga > cancer_sample1_rep1.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample2_rep1.primary_mapped.sorted.bam -bga > cancer_sample2_rep1.depth.bedGraph

# !bedtools genomecov -ibam healthy_control_rep2.primary_mapped.sorted.bam -bga > healthy_control_rep2.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample1_rep2.primary_mapped.sorted.bam -bga > cancer_sample1_rep2.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample2_rep2.primary_mapped.sorted.bam -bga > cancer_sample2_rep2.depth.bedGraph

# !bedtools genomecov -ibam healthy_control_rep3.primary_mapped.sorted.bam -bga > healthy_control_rep3.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample1_rep3.primary_mapped.sorted.bam -bga > cancer_sample1_rep3.depth.bedGraph
# !bedtools genomecov -ibam cancer_sample2_rep3.primary_mapped.sorted.bam -bga > cancer_sample2_rep3.depth.bedGraph

Spočítáme pokrytí v oknech o velikosti 50 000 bází.

In [ ]:
%%bash
samtools faidx Homo_sapiens.GRCh38.dna.toplevel.fa
cut -f1,2 Homo_sapiens.GRCh38.dna.toplevel.fa.fai > chrom.sizes

bedtools makewindows -g chrom.sizes -w 50000 > 50kbps_windows.bed

In [ ]:
%%bash
bedtools coverage -a 50kbps_windows.bed -b healthy_control_rep1.primary_mapped.sorted.bam > healthy_control_rep1.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample1_rep1.primary_mapped.sorted.bam > cancer_sample1_rep1.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample2_rep1.primary_mapped.sorted.bam > cancer_sample2_rep1.coverage_50kbps_windows.tsv

bedtools coverage -a 50kbps_windows.bed -b healthy_control_rep2.primary_mapped.sorted.bam > healthy_control_rep2.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample1_rep2.primary_mapped.sorted.bam > cancer_sample1_rep2.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample2_rep2.primary_mapped.sorted.bam > cancer_sample2_rep2.coverage_50kbps_windows.tsv

bedtools coverage -a 50kbps_windows.bed -b healthy_control_rep3.primary_mapped.sorted.bam > healthy_control_rep3.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample1_rep3.primary_mapped.sorted.bam > cancer_sample1_rep3.coverage_50kbps_windows.tsv
bedtools coverage -a 50kbps_windows.bed -b cancer_sample2_rep3.primary_mapped.sorted.bam > cancer_sample2_rep3.coverage_50kbps_windows.tsv

In [ ]:
use_cols = [0, 1, 2, 3]

healthy_control_rep1_depth_df = pd.read_csv("healthy_control_rep1.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_healthy_control_rep1"])
cancer_sample1_rep1_depth_df = pd.read_csv("cancer_sample1_rep1.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample1_rep1"])
cancer_sample2_rep1_depth_df = pd.read_csv("cancer_sample2_rep1.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample2_rep1"])

healthy_control_rep2_depth_df = pd.read_csv("healthy_control_rep2.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_healthy_control_rep2"])
cancer_sample1_rep2_depth_df = pd.read_csv("cancer_sample1_rep2.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample1_rep2"])
cancer_sample2_rep2_depth_df = pd.read_csv("cancer_sample2_rep2.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample2_rep2"])

healthy_control_rep3_depth_df = pd.read_csv("healthy_control_rep3.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_healthy_control_rep3"])
cancer_sample1_rep3_depth_df = pd.read_csv("cancer_sample1_rep3.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample1_rep3"])
cancer_sample2_rep3_depth_df = pd.read_csv("cancer_sample2_rep3.coverage_50kbps_windows.tsv", header=None, sep="\t", low_memory=False, usecols=use_cols, names=["chrom", "window_start", "window_end", "overlapping_features_cancer_sample2_rep3"])

In [ ]:
merge_on_cols = ["chrom","window_start", "window_end"]
depths_df = healthy_control_rep1_depth_df.merge(cancer_sample1_rep1_depth_df, on=merge_on_cols).merge(cancer_sample2_rep1_depth_df, on=merge_on_cols).merge(healthy_control_rep2_depth_df, on=merge_on_cols).merge(cancer_sample1_rep2_depth_df, on=merge_on_cols).merge(cancer_sample2_rep2_depth_df, on=merge_on_cols).merge(healthy_control_rep3_depth_df, on=merge_on_cols).merge(cancer_sample1_rep3_depth_df, on=merge_on_cols).merge(cancer_sample2_rep3_depth_df, on=merge_on_cols)
depths_df

In [ ]:
# normalizujeme na hloubku sekvenace
# vydělíme tabulku pokrytí počtem čtení
multiply_by = 1_000
depths_df["overlapping_features_healthy_control_rep1"] = (depths_df["overlapping_features_healthy_control_rep1"] / healthy_control_rep1_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample1_rep1"] = (depths_df["overlapping_features_cancer_sample1_rep1"] / cancer_sample1_rep1_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample2_rep1"] = (depths_df["overlapping_features_cancer_sample2_rep1"] / cancer_sample2_rep1_num_reads) * multiply_by

depths_df["overlapping_features_healthy_control_rep2"] = (depths_df["overlapping_features_healthy_control_rep2"] / healthy_control_rep2_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample1_rep2"] = (depths_df["overlapping_features_cancer_sample1_rep2"] / cancer_sample1_rep2_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample2_rep2"] = (depths_df["overlapping_features_cancer_sample2_rep2"] / cancer_sample2_rep2_num_reads) * multiply_by

depths_df["overlapping_features_healthy_control_rep3"] = (depths_df["overlapping_features_healthy_control_rep3"] / healthy_control_rep3_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample1_rep3"] = (depths_df["overlapping_features_cancer_sample1_rep3"] / cancer_sample1_rep3_num_reads) * multiply_by
depths_df["overlapping_features_cancer_sample2_rep3"] = (depths_df["overlapping_features_cancer_sample2_rep3"] / cancer_sample2_rep3_num_reads) * multiply_by

In [ ]:
depths_df

In [ ]:
import plotly.express as px

def plot_chromosome_coverage(chrom_num, df):
    df_subset = df.loc[df["chrom"] == str(chrom_num), ]
    fig = px.line(df_subset, x="window_start", y=["overlapping_features_healthy_control_rep1", "overlapping_features_cancer_sample1_rep1", "overlapping_features_cancer_sample2_rep1", "overlapping_features_healthy_control_rep2", "overlapping_features_cancer_sample1_rep2", "overlapping_features_cancer_sample2_rep2", "overlapping_features_healthy_control_rep3", "overlapping_features_cancer_sample1_rep3", "overlapping_features_cancer_sample2_rep3"], title=f"Pokrytí chromozomu {chrom_num}", line_shape="hv")
    fig.update_layout(xaxis_title="Souřadnice", yaxis_title="Pokrytí")
    fig.show()

In [ ]:
plot_chromosome_coverage(1, depths_df)

In [ ]:
plot_chromosome_coverage(2, depths_df)

In [ ]:
plot_chromosome_coverage(3, depths_df)

In [ ]:
plot_chromosome_coverage(4, depths_df)

In [ ]:
plot_chromosome_coverage(5, depths_df)

In [ ]:
plot_chromosome_coverage(6, depths_df)

In [ ]:
plot_chromosome_coverage(7, depths_df)

In [ ]:
plot_chromosome_coverage(8, depths_df)

In [ ]:
plot_chromosome_coverage(9, depths_df)

In [ ]:
plot_chromosome_coverage(10, depths_df)

In [ ]:
plot_chromosome_coverage(11, depths_df)

In [ ]:
plot_chromosome_coverage(12, depths_df)

In [ ]:
plot_chromosome_coverage(13, depths_df)

In [ ]:
plot_chromosome_coverage(14, depths_df)

In [ ]:
plot_chromosome_coverage(15, depths_df)

In [ ]:
plot_chromosome_coverage(16, depths_df)

In [ ]:
plot_chromosome_coverage(17, depths_df)

In [ ]:
plot_chromosome_coverage(18, depths_df)

In [ ]:
plot_chromosome_coverage(19, depths_df)

In [ ]:
plot_chromosome_coverage(20, depths_df)

In [ ]:
plot_chromosome_coverage(21, depths_df)

In [ ]:
plot_chromosome_coverage(22, depths_df)

In [ ]:
plot_chromosome_coverage("X", depths_df)

In [ ]:
plot_chromosome_coverage("Y", depths_df)